In [1]:
import pandas as pd
import janitor
import numpy as np

## Renewable capacity

In [23]:
generation_gva = pd.read_excel('renewable_capacity.xls', sheet_name='Trends', skiprows=6).clean_names()[:15]
generation_gva = generation_gva[generation_gva.region.isin(['North East', 'UK Total'])]

In [29]:
generation_gva

,region,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
3,North East,80.9,112.3,107.6,112.0,154.600,172.671400,246.716600,284.051588,332.408188,449.899668,661.144158,711.668618,826.978237,920.359378,1067.056158,1551.772302,1556.764076,1561.306556,1567.507346
14,UK Total,3458.7,3771.8,4534.0,5031.7,5745.568,6836.901207,8004.189187,9255.875526,12381.250943,15648.592973,19960.876958,24919.540518,30965.664201,35650.741418,40292.553133,44128.711363,46971.249552,47917.715001,49702.270500


In [57]:
long_generation = generation_gva.set_index('region').melt(ignore_index=False).reset_index().rename(columns = {'variable':'year'})

In [59]:
long_generation.to_csv('economic_energy_generation_capacity.csv')

## Gas and electric use

In [11]:
subnational_electric = pd.read_excel('subnational_electricity_consumption_statistics_2005-2020.xlsx', sheet_name=None, skiprows=4)
subnational_gas = pd.read_excel('subnational_gas_consumption_statistics_2005-2020.xlsx', sheet_name=None, skiprows=5)

In [12]:
electric_years = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',  '2020']
gas_years = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',  '2020']

In [13]:
df = pd.DataFrame(columns=['region', 'year', 'type','value'])

for ix, year in enumerate(electric_years):
    elec_year = subnational_electric[year].clean_names()
    gas_year = subnational_gas[year].clean_names()
    nat_elec = elec_year[(elec_year.local_authority == 'All local authorities') & (elec_year.country_or_region.str.contains('Great Britain'))][['mean_consumption\n_kwh_per_meter_\nall_meters']].values[0,0]
    ne_elec = elec_year[(elec_year.local_authority == 'All local authorities') & (elec_year.country_or_region.str.contains('North East'))][['mean_consumption\n_kwh_per_meter_\nall_meters']].values[0,0]
    nat_gas = gas_year[(gas_year.local_authority == 'All local authorities') & (gas_year.country_or_region.str.contains('Great Britain'))][['mean_consumption\n_kwh_per_meter_\nall_meters']].values[0,0]
    ne_gas = gas_year[(gas_year.local_authority == 'All local authorities') & (gas_year.country_or_region.str.contains('North East'))][['mean_consumption\n_kwh_per_meter_\nall_meters']].values[0,0]
    
    df = pd.concat([df, pd.DataFrame.from_records([{'region': 'gb', 'year':year, 'type':'electric', 'value':nat_elec}])])
    df = pd.concat([df, pd.DataFrame.from_records([{'region':'ne', 'year':year, 'type':'electric', 'value':ne_elec}])])
    df = pd.concat([df, pd.DataFrame.from_records([{'region':'gb', 'year':year, 'type':'gas', 'value':nat_gas}])])
    df = pd.concat([df, pd.DataFrame.from_records([{'region':'ne', 'year':year, 'type':'gas', 'value':ne_gas}])])

df.reset_index(inplace=True, drop=True)

In [ ]:
df = df.replace(['gb','ne'], ['GB Total', 'North East']).replace(['electric', 'gas'], ['Electric', 'Gas'])

In [65]:
df.to_csv('economic_energy_gas_electric.csv')

## Total use by type

In [2]:
subnational_total = pd.read_excel('subnational_total_final_energy_consumption_2019.xlsx', sheet_name=None, skiprows=6)

In [3]:
total_years = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018r', '2019r']

In [4]:
df_total = pd.DataFrame(columns=['region', 'year', 'type','value'])

for ix, year in enumerate(total_years):
    total_year = subnational_total[year].clean_names()
    nat_domestic = total_year[(total_year.local_authority == 'All local authorities') & (total_year.country_or_region.str.contains('Great Britain'))][['all_fuels_\ndomestic']].values[0,0]
    nat_transport = total_year[(total_year.local_authority == 'All local authorities') & (total_year.country_or_region.str.contains('Great Britain'))][['all_fuels_\ntransport\n[note1]']].values[0,0]
    nat_commercial = total_year[(total_year.local_authority == 'All local authorities') & (total_year.country_or_region.str.contains('Great Britain'))][['all_fuels_\nindustrial_\ncommercial\nand_other']].values[0,0]
    ne_domestic = total_year[(total_year.local_authority == 'All local authorities') & (total_year.country_or_region.str.contains('North East'))][['all_fuels_\ndomestic']].values[0,0]
    ne_transport = total_year[(total_year.local_authority == 'All local authorities') & (total_year.country_or_region.str.contains('North East'))][['all_fuels_\ntransport\n[note1]']].values[0,0]
    ne_commercial = total_year[(total_year.local_authority == 'All local authorities') & (total_year.country_or_region.str.contains('North East'))][['all_fuels_\nindustrial_\ncommercial\nand_other']].values[0,0]

    df_total = pd.concat([df_total, pd.DataFrame.from_records([{'region': 'gb', 'year':year.strip('r'), 'type':'domestic', 'value':nat_domestic}])])
    df_total = pd.concat([df_total, pd.DataFrame.from_records([{'region': 'gb', 'year':year.strip('r'), 'type':'transport', 'value':nat_transport}])])
    df_total = pd.concat([df_total, pd.DataFrame.from_records([{'region': 'gb', 'year':year.strip('r'), 'type':'commercial', 'value':nat_commercial}])])
    df_total = pd.concat([df_total, pd.DataFrame.from_records([{'region': 'ne', 'year':year.strip('r'), 'type':'domestic', 'value':ne_domestic}])])
    df_total = pd.concat([df_total, pd.DataFrame.from_records([{'region': 'ne', 'year':year.strip('r'), 'type':'transport', 'value':ne_transport}])])
    df_total = pd.concat([df_total, pd.DataFrame.from_records([{'region': 'ne', 'year':year.strip('r'), 'type':'commercial', 'value':ne_commercial}])])


df_total.reset_index(inplace=True, drop=True)

In [8]:
df_total = df_total.replace(['gb','ne'], ['GB Total', 'North East']).replace(['domestic', 'transport', 'commercial'], ['Domestic', 'Transport', 'Commercial'])

In [10]:
df_total.to_csv('economic_energy_total.csv')